In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import astropy.io.fits as pyfits
import astropy.utils as autils
import requests
import json
from pprint import pprint as pp 
import os

In [2]:
# define database search
natroot = "https://astroarchive.noirlab.edu"
adsurl = f'{natroot}/api/adv_search'
adsurl


'https://astroarchive.noirlab.edu/api/adv_search'

In [3]:
jj = {
    "outfields" : [
        "md5sum",
        "ra_min",  # "ra" is a "range_field" in Postgres, access either part of range
        "archive_filename",
        "instrument",
        "proc_type",
        "obs_type",
        "release_date",
        "proposal",
        "caldat",
        "EXPNUM",    # AUX field. Slows search
        "AIRMASS",
    ],
    "search" : [
        ["instrument", "decam"],
        ["proc_type", "instcal"],
        ["prod_type", "image", "dqmask"],
        ["proposal", "2022B-297190"],
        ["obs_type", "object"],
        
    ]
}
apiurl = f'{adsurl}/find/?limit=10'
print(f'Using API url: {apiurl}')
objects = pd.DataFrame(requests.post(apiurl,json=jj).json()[1:])
objects

Using API url: https://astroarchive.noirlab.edu/api/adv_search/find/?limit=10


,obs_type,release_date,caldat,proposal,proc_type,md5sum,archive_filename,instrument,ra_min,EXPNUM,AIRMASS
0,object,2022-12-27,2022-12-27,2022B-297190,instcal,573360f3d1d4de3a68d71b43e3d6d60b,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,45.156917,1160129,1.14
1,object,2022-12-27,2022-12-27,2022B-297190,instcal,ad48a37ccdcccc91b418ee581d482978,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,45.162708,1160133,1.14
2,object,2022-12-27,2022-12-27,2022B-297190,instcal,abe6c876a8491e030876d8b022305840,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,45.162708,1160133,1.14
3,object,2022-12-27,2022-12-27,2022B-297190,instcal,a93a04905c56d1a1cc8fc97f5b110fb5,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,55.059625,1160148,1.12
4,object,2022-12-27,2022-12-27,2022B-297190,instcal,4ea049d091c3161faf7d88c0ecc936ff,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,167.012625,1160307,1.18
5,object,2022-12-27,2022-12-27,2022B-297190,instcal,a552ccbd74f246fab9e58a899d2cdf97,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,167.012625,1160307,1.18
6,object,2022-12-27,2022-12-27,2022B-297190,instcal,f4768c7fe86351b09d30304e27d8e9bd,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,162.551750,1160292,1.21
7,object,2022-12-27,2022-12-27,2022B-297190,instcal,a3748a81e3efcf81781f03acfe6105df,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,164.337667,1160298,1.20
8,object,2022-12-27,2022-12-27,2022B-297190,instcal,45e5d73a37a3f0df65de70bf639d8ae0,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,55.059625,1160148,1.12
9,object,2022-12-27,2022-12-27,2022B-297190,instcal,f2e40acd187a263c001f2c11fb6f9c10,/net/archive/pipe/20221227/ct4m/2022B-297190/c...,decam,167.949625,1160310,1.20


In [9]:
fileids = objects['md5sum']

for i,fileurl in enumerate(fileids):

    fileurl = f'{natroot}/api/retrieve/{fileids[i]}'
    sbatch_text = """#!/bin/bash \
    #SBATCH --qos=debug \
    #SBATCH --time=5 \
    #SBATCH --nodes=1 \
    #SBATCH --tasks-per-node=1 \
    #SBATCH --output="""+str(i)+""".log \
    #SBATCH --constraint=haswell \

    srun wget -nv --content-disposition """+fileurl

    np.savetxt("sbatch_"+str(i)+".sh",[sbatch_text], fmt="%s")

In [36]:
#Now you should be able to send the downloads with sbatch as batch runs, but here is a test for just one file:

os.system("wget -nv --content-disposition "+fileurl)

2022-10-27 16:25:05 URL:https://astroarchive.noirlab.edu/api/retrieve/afff9dc389784c011b45a69734073da8/ [317856960/317856960] -> "c4d_221026_072021_ooi_z_desirt.fits.fz" [1]


0